In [1]:
import sqlite3
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv

In [2]:
connection = sqlite3.connect('../chembl_29_sqlite (1)/chembl_29_sqlite/chembl_30.db')
cursor = connection.cursor()
#old_query = 'SELECT ASY.CHEMBL_ID, ASY.CELL_ID, ACT.STANDARD_VALUE,ACT.STANDARD_TYPE,ACT.STANDARD_UNITS FROM ASSAYS ASY, ACTIVITIES ACT WHERE ACT.ASSAY_ID = ASY.ASSAY_ID'
query = "SELECT ACT.ACTIVITY_ID, ASY.ASSAY_ID, ACT.MOLREGNO , ASY.CELL_ID, ACT.STANDARD_VALUE,ACT.STANDARD_UNITS, ACT.STANDARD_TYPE " \
    " FROM ACTIVITIES ACT INNER JOIN ASSAYS ASY ON ACT.ASSAY_ID = ASY.ASSAY_ID" \
    " WHERE ASY.CELL_ID IS NOT NULL"

In [3]:
cursor.execute(query)

In [4]:
activity_dictionary = cursor.fetchall()
#SELECT ACT.ACTIVITY_ID, ASY.ASSAY_ID, ACT.MOLREGNO , ASY.CELL_ID, ACT.STANDARD_VALUE,ACT.STANDARD_UNITS, ACT.STANDARD_TYPE 
activity_df = pd.DataFrame(activity_dictionary,columns=['activity_id','assay_id','compound_id','cell_id','std_value','std_units','std_type'])
activity_df

,activity_id,assay_id,compound_id,cell_id,std_value,std_units,std_type
0,161152,5,556,163,600000.0,nM,CC50
1,162404,5,82983,163,90000.0,nM,CC50
2,164804,5,1633207,163,340000.0,nM,CC50
3,166046,5,1633675,163,2400.0,nM,CC50
4,166057,5,47675,163,7000000.0,nM,CC50
...,...,...,...,...,...,...,...
5987118,23119972,2095877,2531656,594,50.0,nM,EC50
5987119,23119973,2095878,2531656,1213,190.0,nM,EC50
5987120,23119978,2095878,2537326,1213,18.0,µM,EC50
5987121,23119974,2095879,2531656,630,370.0,nM,EC50


In [5]:
raw_act = activity_df[activity_df['std_type'].isin(['IC50','CC50','EC50','GI50'])]
raw_act

,activity_id,assay_id,compound_id,cell_id,std_value,std_units,std_type
0,161152,5,556,163,600000.0,nM,CC50
1,162404,5,82983,163,90000.0,nM,CC50
2,164804,5,1633207,163,340000.0,nM,CC50
3,166046,5,1633675,163,2400.0,nM,CC50
4,166057,5,47675,163,7000000.0,nM,CC50
...,...,...,...,...,...,...,...
5987118,23119972,2095877,2531656,594,50.0,nM,EC50
5987119,23119973,2095878,2531656,1213,190.0,nM,EC50
5987120,23119978,2095878,2537326,1213,18.0,µM,EC50
5987121,23119974,2095879,2531656,630,370.0,nM,EC50


In [6]:
raw_act = raw_act[raw_act['std_units'].isin(['nM', '10^4M', '/uM', "10'-11uM", "10'10uM", "10'8pM", "10'7pM", "10'6pM", "10'5pM", "10'-4nM", "10'6uM", "10'5uM", 'µM'])]
raw_act

,activity_id,assay_id,compound_id,cell_id,std_value,std_units,std_type
0,161152,5,556,163,600000.0,nM,CC50
1,162404,5,82983,163,90000.0,nM,CC50
2,164804,5,1633207,163,340000.0,nM,CC50
3,166046,5,1633675,163,2400.0,nM,CC50
4,166057,5,47675,163,7000000.0,nM,CC50
...,...,...,...,...,...,...,...
5987118,23119972,2095877,2531656,594,50.0,nM,EC50
5987119,23119973,2095878,2531656,1213,190.0,nM,EC50
5987120,23119978,2095878,2537326,1213,18.0,µM,EC50
5987121,23119974,2095879,2531656,630,370.0,nM,EC50


In [7]:
#TRANSFORMACIONES A uM

# nM
raw_act.loc[ raw_act['std_units'] == 'nM',['std_value']] * 0.001
raw_act.loc[ raw_act['std_units'] == 'nM',['std_units']] = 'uM'

# 10^4M 
raw_act.loc[ raw_act['std_units'] == '10^4M',['std_value']] * 10**4
raw_act.loc[ raw_act['std_units'] == '10^4M',['std_value']] * 1000000
raw_act.loc[ raw_act['std_units'] == '10^4M',['std_units']] = 'uM'

# /uM 
raw_act.loc[ raw_act['std_units'] == '/uM',['std_units']] = 'uM'


# 10'-11uM 
raw_act.loc[ raw_act['std_units'] == "10'-11uM",['std_value']] * 10**-11
raw_act.loc[ raw_act['std_units'] == "10'-11uM",['std_units']] = 'uM'

# 10'10uM
raw_act.loc[ raw_act['std_units'] == "10'10uM",['std_value']] * 10**10
raw_act.loc[ raw_act['std_units'] == "10'10uM",['std_units']] = 'uM'

# 10'8pM  
raw_act.loc[ raw_act['std_units'] == "10'8pM",['std_value']] * 10**8
raw_act.loc[ raw_act['std_units'] == "10'8pM",['std_value']] * 1000000
raw_act.loc[ raw_act['std_units'] == "10'8pM",['std_units']] = 'uM'

# 10'7pM
raw_act.loc[ raw_act['std_units'] == "10'7pM",['std_value']] * 10**7
raw_act.loc[ raw_act['std_units'] == "10'7pM",['std_value']] * 0.000001
raw_act.loc[ raw_act['std_units'] == "10'7pM",['std_units']] = 'uM'

#[   "10'5uM", 'µM']

# 10'6pM 
raw_act.loc[ raw_act['std_units'] == "10'6pM",['std_value']] * 10**6
raw_act.loc[ raw_act['std_units'] == "10'6pM",['std_value']] * 0.000001
raw_act.loc[ raw_act['std_units'] == "10'6pM",['std_units']] = 'uM'

# 10'5pM    
raw_act.loc[ raw_act['std_units'] == "10'5pM",['std_value']] * 10**5
raw_act.loc[ raw_act['std_units'] == "10'5pM",['std_value']] * 0.000001
raw_act.loc[ raw_act['std_units'] == "10'5pM",['std_units']] = 'uM'

# 10'-4nM  
raw_act.loc[ raw_act['std_units'] == "10'-4nM",['std_value']] * 10**-4
raw_act.loc[ raw_act['std_units'] == "10'-4nM",['std_value']] * 0.001
raw_act.loc[ raw_act['std_units'] == "10'-4nM",['std_units']] = 'uM'


# 10'6uM  
raw_act.loc[ raw_act['std_units'] == "10'6uM",['std_value']] * 10**6
raw_act.loc[ raw_act['std_units'] == "10'6uM",['std_units']] = 'uM'

# 10'5uM
raw_act.loc[ raw_act['std_units'] == "10'5uM",['std_value']] * 10**5
raw_act.loc[ raw_act['std_units'] == "10'5uM",['std_units']] = 'uM'

# µM 
raw_act.loc[ raw_act['std_units'] == "µM",['std_units']] = 'uM'

In [9]:
raw_act

,activity_id,assay_id,compound_id,cell_id,std_value,std_units,std_type
0,161152,5,556,163,600000.0,uM,CC50
1,162404,5,82983,163,90000.0,uM,CC50
2,164804,5,1633207,163,340000.0,uM,CC50
3,166046,5,1633675,163,2400.0,uM,CC50
4,166057,5,47675,163,7000000.0,uM,CC50
...,...,...,...,...,...,...,...
5987118,23119972,2095877,2531656,594,50.0,uM,EC50
5987119,23119973,2095878,2531656,1213,190.0,uM,EC50
5987120,23119978,2095878,2537326,1213,18.0,uM,EC50
5987121,23119974,2095879,2531656,630,370.0,uM,EC50


In [10]:
raw_act['active']= np.where(raw_act['std_value']<=10.0,1,-1)

C:\Users\Reyum\AppData\Local\Temp\ipykernel_18136\191682871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_act['active']= np.where(raw_act['std_value']<=10.0,1,-1)


In [11]:
raw_act

,activity_id,assay_id,compound_id,cell_id,std_value,std_units,std_type,active
0,161152,5,556,163,600000.0,uM,CC50,-1
1,162404,5,82983,163,90000.0,uM,CC50,-1
2,164804,5,1633207,163,340000.0,uM,CC50,-1
3,166046,5,1633675,163,2400.0,uM,CC50,-1
4,166057,5,47675,163,7000000.0,uM,CC50,-1
...,...,...,...,...,...,...,...,...
5987118,23119972,2095877,2531656,594,50.0,uM,EC50,-1
5987119,23119973,2095878,2531656,1213,190.0,uM,EC50,-1
5987120,23119978,2095878,2537326,1213,18.0,uM,EC50,-1
5987121,23119974,2095879,2531656,630,370.0,uM,EC50,-1


In [11]:
raw_act.to_csv('../csvs/raw_activity_30.csv',header=True,index=True)

In [12]:
act = pd.DataFrame(raw_act[['compound_id','cell_id','active']])
act

,compound_id,cell_id,active
0,556,163,-1
1,82983,163,-1
2,1633207,163,-1
3,1633675,163,-1
4,47675,163,-1
...,...,...,...
5987118,2531656,594,-1
5987119,2531656,1213,-1
5987120,2537326,1213,-1
5987121,2531656,630,-1


In [13]:
#act.drop_duplicates()
act.to_csv('../csvs/summary30.csv',header=True,index=True)

In [14]:
act=pd.read_csv('../csvs/summary30.csv',index_col=0).drop_duplicates(keep='first')

In [15]:
df_count = act.groupby(by=['compound_id','cell_id'])
df_count.size()

compound_id  cell_id
10           5566       1
23           330        1
             555        1
38           5566       1
39           5566       1
                       ..
2537375      5673       1
2537376      303        1
             646        1
             721        1
             786        1
Length: 3376048, dtype: int64

In [16]:
df_count.size()[df_count.size()==1]

compound_id  cell_id
10           5566       1
23           330        1
             555        1
38           5566       1
39           5566       1
                       ..
2537375      5673       1
2537376      303        1
             646        1
             721        1
             786        1
Length: 3359808, dtype: int64

In [17]:
index1 = pd.MultiIndex.from_arrays([act[col] for col in ['compound_id', 'cell_id']])
index2 = df_count.size()[df_count.size()==1].index

In [18]:
summ_act=act.loc[index1.isin(index2)]
summ_act


,compound_id,cell_id,active
0,556,163,-1
1,82983,163,-1
2,1633207,163,-1
3,1633675,163,-1
4,47675,163,-1
...,...,...,...
5987118,2531656,594,-1
5987119,2531656,1213,-1
5987120,2537326,1213,-1
5987121,2531656,630,-1


In [19]:
summ_act.to_csv('../csvs/act_summary30.csv',header=True,index=True)

In [20]:
#Conteo de -1 y 1 

size = summ_act.shape[0]
print (len(summ_act[summ_act['active'].isin([1])])/size)
print (len(summ_act[summ_act['active'].isin([-1])])/size)

0.0208723236565899
0.9791276763434101


SUMMARY

In [21]:
summary = pd.read_csv('../csvs/act_summary29.csv',index_col=0)
summary

FileNotFoundError: [Errno 2] No such file or directory: '../csvs/act_summary29.csv'

In [ ]:
#cells that report activities and inactivities
cell_in_act = summary.groupby(by=['cell_id','active']).size().groupby(by=['cell_id']).size()[summary.groupby(by=['cell_id','active']).size().groupby(by=['cell_id']).size()==2].index
summary[summary['cell_id'].isin(cell_in_act)]

In [ ]:
b_summary_cell = summary[summary['cell_id'].isin(cell_in_act)]
b_summary_cell

In [ ]:
b_summary_cell.groupby(by=['cell_id']).size().max()

In [ ]:
b_summary_cell.groupby(by=['cell_id']).size().min()

In [ ]:
#count_in_act_cell = b_summary.groupby(by=['cell_id','active']).size()
count_in_act_cell = b_summary_cell.groupby(by=['cell_id','active'])
count_in_act_cell

In [ ]:
x_cell=count_in_act_cell['active'].size().values
x_cell

In [ ]:
z_cell=x_cell[::2]
w_cell=x_cell[1::2]
plt.xlim(0,600)
plt.ylim(0,600)
plt.scatter(x=z_cell,y=w_cell)
plt.show()

INTERSECCIONES

In [ ]:
intersections_matrix = np.zeros((952,952))

In [ ]:
cell_rad_df = pd.read_csv('../datos/cell_c_radii_df.csv',index_col=0)
cell_rad_df

In [ ]:
def distancia(punto_a,punto_b):
    
    resta = list(map(lambda x,y: x-y ,punto_a,punto_b))
    vector = np.array(resta) 
    return mt.sqrt(sum(pow(element, 2) for element in vector))

In [ ]:
for cell_a in cell_rad_df.iterrows():
    cell_a_centroid =  eval(cell_a[1][1])
    cell_r1 = cell_a[1][2]
    print(cell_a[1][0])
    for cell_b in cell_rad_df.iterrows():
        if cell_a[0]!=cell_b[0]:
            cell_b_centroid =  eval(cell_b[1][1])
            cell_r2 = cell_b[1][2]
            dist_cell =distancia(cell_a_centroid,cell_b_centroid)
            radios =cell_r1 + cell_r2 
            intersec = (radios - dist_cell) / radios
            intersections_matrix[cell_a[0]][cell_b[0]] =intersec
        else:
            intersections_matrix[cell_a[0]][cell_b[0]] = 0

In [ ]:
df_inter = pd.read_csv('intersecciones_div_rad.csv',index_col=0)
df_inter

In [ ]:
#calculo mayor valor
maximo_val = df_inter.max() #encuentra el maximo de cada columna, en este caso las columnas son las celulas
maximos =maximo_val.to_frame()
maximos.nlargest(1,0)

In [ ]:
minimo_val = df_inter.min()
minimos =minimo_val.to_frame()
minimos.nsmallest(1,0)

In [ ]:
nombres= cell_rad_df['cell_name']
df_inter_cell = pd.concat([nombres, df_inter], axis=1)
df_inter_cell

In [ ]:

df_mapeo_cell = pd.read_csv('../datos/cell_mapping.csv',index_col=0)
df_mapeo_cell

In [ ]:
#top 10 con mapeo a cell_id

lista= []

#Se recorre el dataframe  horizontalmente
for label,content  in df_inter_cell.items():
    if label != 'cell_name':
        
        
        id = int(label)
        #Obtenemos el nombre de la linea celular actual con la ayuda del label recorrido
        name = df_inter_cell['cell_name'][id]

        #Otenemos el top 10
        top10 = df_inter_cell.nlargest(10,str(id))[['cell_name',str(id)]]

        #Guardamos en una lista los nombres de su top3 
        lista_top10 = top10['cell_name'].tolist()
        

        #Realizamos un cutoff para los valores mayyores a 0.8 

        lista_val_top10 = [val for val in top10[label].tolist() if val >= 0.8]

        lista_top10= lista_top10[:len(lista_val_top10)]

       
       #Si ninguna linea celular del top10 supero el cutoff no se la añade 

        if len(lista_top10) !=0:
            #----------------------------------------

            #Realizamos el mapeo 

            df_name_map = df_mapeo_cell[df_mapeo_cell['cellosaurus_id'].isin([name])]

            #si encuentra que existe la celula actual en el mapeo entonces se lo agrega
            if  not df_name_map.empty:
                
                cells =[]
                values=[]                
                
                for index,cell_name in enumerate(lista_top10):
                    
                    df_name_map_top10 = df_mapeo_cell[df_mapeo_cell['cellosaurus_id'].isin([cell_name])]
                    
                    if not df_name_map_top10.empty:
                        cells.append(df_name_map_top10['cell_id'][0])
                        values.append(lista_val_top10[index])
                        
                lista.append([df_name_map['cell_id'][0],cells,values])

In [ ]:
#TOP10
listas_top10_mappings =pd.DataFrame(lista, columns=['cell_id','cell_top10','value_top10'])
listas_top10_mappings.to_csv('listas_top10_mapping.csv',index=0)

In [ ]:
lista_mapp = pd.read_csv('listas_top10_mapping.csv')
lista_mapp

In [ ]:
#top 3 con mapeo a cell_id

lista= []

#Se recorre el dataframe  horizontalmente
for label,content  in df_inter_cell.items():
    if label != 'cell_name':
        
        
        id = int(label)
        #Obtenemos el nombre de la linea celular actual con la ayuda del label recorrido
        name = df_inter_cell['cell_name'][id]

        #Otenemos el top 10
        top10 = df_inter_cell.nlargest(3,str(id))[['cell_name',str(id)]]

        #Guardamos en una lista los nombres de su top3 
        lista_top10 = top10['cell_name'].tolist()
        

        #Realizamos un cutoff para los valores mayyores a 0.8 

        lista_val_top10 = [val for val in top10[label].tolist() if val >= 0.8]

        lista_top10= lista_top10[:len(lista_val_top10)]

       
       #Si ninguna linea celular del top10 supero el cutoff no se la añade 

        if len(lista_top10) !=0:
            #----------------------------------------

            #Realizamos el mapeo 

            df_name_map = df_mapeo_cell[df_mapeo_cell['cellosaurus_id'].isin([name])]

            #si encuentra que existe la celula actual en el mapeo entonces se lo agrega
            if  not df_name_map.empty:
                
                cells =[]
                values=[]                
                
                for index,cell_name in enumerate(lista_top10):
                    
                    df_name_map_top10 = df_mapeo_cell[df_mapeo_cell['cellosaurus_id'].isin([cell_name])]
                    
                    if not df_name_map_top10.empty:
                        cells.append(df_name_map_top10['cell_id'][0])
                        values.append(lista_val_top10[index])
                        
                lista.append([df_name_map['cell_id'][0],cells,values])

In [ ]:
#TOP3
listas_top10_mappings =pd.DataFrame(lista, columns=['cell_id','cell_top10','value_top10'])
listas_top10_mappings.to_csv('listas_top3_mapping.csv',index=0)

In [ ]:
lista_mapp = pd.read_csv('listas_top3_mapping.csv')
lista_mapp

In [ ]:
#top 5 con mapeo a cell_id

lista= []

#Se recorre el dataframe  horizontalmente
for label,content  in df_inter_cell.items():
    if label != 'cell_name':
        
        
        id = int(label)
        #Obtenemos el nombre de la linea celular actual con la ayuda del label recorrido
        name = df_inter_cell['cell_name'][id]

        #Otenemos el top 10
        top10 = df_inter_cell.nlargest(5,str(id))[['cell_name',str(id)]]

        #Guardamos en una lista los nombres de su top3 
        lista_top10 = top10['cell_name'].tolist()
        

        #Realizamos un cutoff para los valores mayyores a 0.8 

        lista_val_top10 = [val for val in top10[label].tolist() if val >= 0.8]

        lista_top10= lista_top10[:len(lista_val_top10)]

       
       #Si ninguna linea celular del top10 supero el cutoff no se la añade 

        if len(lista_top10) !=0:
            #----------------------------------------

            #Realizamos el mapeo 

            df_name_map = df_mapeo_cell[df_mapeo_cell['cellosaurus_id'].isin([name])]

            #si encuentra que existe la celula actual en el mapeo entonces se lo agrega
            if  not df_name_map.empty:
                
                cells =[]
                values=[]                
                
                for index,cell_name in enumerate(lista_top10):
                    
                    df_name_map_top10 = df_mapeo_cell[df_mapeo_cell['cellosaurus_id'].isin([cell_name])]
                    
                    if not df_name_map_top10.empty:
                        cells.append(df_name_map_top10['cell_id'][0])
                        values.append(lista_val_top10[index])
                        
                lista.append([df_name_map['cell_id'][0],cells,values])

In [ ]:
#TOP5
listas_top10_mappings =pd.DataFrame(lista, columns=['cell_id','cell_top10','value_top10'])
listas_top10_mappings.to_csv('listas_top5_mapping.csv',index=0)

In [ ]:
lista_mapp = pd.read_csv('listas_top5_mapping.csv')
lista_mapp

In [ ]:
#Estos datos seran utilizados para la prediccion

In [ ]:
#guardado en prediccion_activos_4.py
import math as mt 
from numpy import random
import pandas as pd
import numpy as np
lista_mapp = pd.read_csv('listas_top10_mapping.csv')
summary = pd.read_csv('summary29.csv',index_col=0)
compounds_pred = summary[summary['cell_id'].isin(lista_mapp['cell_id'].tolist())]
compounds_pred_sim = compounds_pred.drop_duplicates(subset=['compound_id','cell_id','active'], keep="first")

compuestos_val = compounds_pred_sim.compound_id.unique()
lista_respuesta = []
lista_problemas = []
for comp in compuestos_val:
    #ahora necesitamos extrar las celulas que tienen interacciones con este compuesto
    cell_interact_compound_c = compounds_pred_sim[compounds_pred_sim['compound_id'].isin([comp])]
    cell_interact_compound = cell_interact_compound_c.drop_duplicates(subset=['compound_id','cell_id'], keep="first")



    lista_cell_interact_compound = cell_interact_compound['cell_id'].to_list()
    lista_cell_interact_compound_active = cell_interact_compound['active'].to_list()
        

    for index, cell_interact in cell_interact_compound.iterrows():
        #print(cell_interact['active'])
        
        #ahora ver si el top 10  celulas tienen interaccion con el compuesto

        #Esto para una celula que interactua
        df_cell_interact_top10 = lista_mapp[lista_mapp['cell_id'].isin([cell_interact['cell_id']])]
        #cell_top10 =eval(df_cell_interact_top10['cell_top10'].tolist()[0])
        #val_top10 =eval(df_cell_interact_top10['value_top10'].tolist()[0])
        
        lista_actividades=[]

        active_prediccion = cell_interact['active']
        #print(cell_interact['cell_id'])
        
        #    df_cell_interact = lista_mapp[lista_mapp['cell_id'].isin([cell_interact])]
        cell_top10 =eval(df_cell_interact_top10['cell_top10'].tolist()[0])
        #val_top10 =eval(df_cell_interact_top10['value_top10'].tolist()[0])
        #    lista_actividades=[]

        #    active_prediccion = 

        for cell in cell_top10:

        #Debido a que solo se obtiene un dato en el dataframe, el cual contiene las listas 
        #no es posible
        #for index, cell in df_cell_interact_top10.iterrows():
            #Probar si la cell actual tiene doble actividad
            #En el caso que exista doble actividad, se agregaran tanto en la lista de predicciones
            #como en la lista de problemas  cell_interact['cell_id']
            #cell_interact id existe en  cell_interact_compound_c

            cell_double_act = cell_interact_compound_c[cell_interact_compound_c['cell_id'].isin([cell_interact['cell_id']])]
            
            if len(cell_double_act)>1:
                #en este caso la celula que interactura tiene actividad y no actividad
                #print("DOBLEEEE")
                lista_problemas.append([82983,cell_interact['cell_id'],cell,active_prediccion,"cell-m-d"])
                lista_respuesta.append([82983,cell_interact['cell_id'],cell,active_prediccion])
            else:


            #print(cell)
            #Comprobamos si la cell actual del top 10 ya tiene relacion con el compuesto

                cell_exist = cell_interact_compound_c[cell_interact_compound_c['cell_id'].isin([cell])]
                
                
                    
                if len(cell_exist) ==1:
                    #print("--",cell_exist['active'].values[0])
                    #print("++",active_prediccion)
                    #Existe mapeo para esa cell
                    #Analizamos si la actividad que existe es diferente a la que se tiene
                    #aaa =cell_exist['active'].values[0]

                    if cell_exist['active'].values[0]!=active_prediccion:
                        #print("Existen problemas ",cell_exist['active'].values[0]," ", active_prediccion )
                        #print(cell_exist)
                        #print(cell_exist)
                        lista_problemas.append([82983,cell_interact['cell_id'],cell,active_prediccion,"cell-t-e"])
                        lista_respuesta.append([82983,cell_interact['cell_id'],cell,active_prediccion])
                    else:
                        #print("No existen problemas, se agrega")        
                        lista_respuesta.append([comp,cell_interact['cell_id'],cell,active_prediccion])

                elif len(cell_exist) >1:
                    # Existe el compuesto pero tiene 2 interacciones
                    lista_problemas.append([comp,cell_interact['cell_id'],cell,active_prediccion,"cell-t-d"])
                    lista_respuesta.append([comp,cell_interact['cell_id'],cell,active_prediccion])

                else:
                
                    lista_respuesta.append([82983,cell_interact['cell_id'],cell,active_prediccion])



df_recomendacion = pd.DataFrame(lista_respuesta, columns =['compuesto','cell_interact','cell','active'])
df_recomendacion.to_csv('NEW_recomendaciones.csv',index=0)

df_problemas = pd.DataFrame(lista_problemas, columns =['compuesto','cell_interact','cell','active','problema'])
df_problemas.to_csv('problemas.csv',index=0)



In [ ]:
df_rec = pd.read_csv("NEW_recomendaciones.csv")
df_rec

In [ ]:
df_err = pd.read_csv("problemas.csv")
df_err

In [ ]:
df_err.problema.unique()

'cell-m-d' # En este caso la celula que interactura tiene actividad y no actividad
'cell-t-e' # Tiene una interaccion contraria a la que se esta prediciendo.
'cell-t-d' # Existe el compuesto pero tiene 2 interacciones